In [1]:
import boto3
import json
from jsonschema import validate
from elasticsearch import Elasticsearch, RequestsHttpConnection
from elasticsearch.helpers import scan
from requests_aws4auth import AWS4Auth

region = 'us-east-1'
service = 'es'
eshost = 'search-world-modelers-dev-gjvcliqvo44h4dgby7tn3psw74.us-east-1.es.amazonaws.com'

session = boto3.Session(region_name=region, profile_name='wmuser')
credentials = session.get_credentials()
credentials = credentials.get_frozen_credentials()
access_key = credentials.access_key
secret_key = credentials.secret_key
token = credentials.token

aws_auth = AWS4Auth(
    access_key,
    secret_key,
    region,
    service,
    session_token=token
)


In [2]:
es = Elasticsearch(
    hosts = [{'host': eshost, 'port': 443}],
    http_auth=aws_auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

In [3]:
index = "wm-dev"

In [4]:
query = {
    "aggs" : {
        "categories" : {
            "terms" : { "field" : "category.keyword" } 
        }
    }
}

categories = es.search(index=index, body=query)['aggregations']['categories']['buckets']

In [5]:
categories

[{'key': 'Twelve-Month Eval Docs', 'doc_count': 473},
 {'key': 'November 2019 SSudan Docs', 'doc_count': 405},
 {'key': 'November 2019 Ethiopia Docs', 'doc_count': 255},
 {'key': 'Six-Month Evaluation Documents', 'doc_count': 38},
 {'key': 'Additional Six-Month Eval Docs', 'doc_count': 29},
 {'key': 'Luma-Provided Ethiopia Docs', 'doc_count': 21},
 {'key': 'Migration', 'doc_count': 19}]

In [14]:
results = []

cats = ['Twelve-Month Eval Docs','Six-Month Evaluation Documents','Additional Six-Month Eval Docs'],

for cat in cats:
    query = {
        "query" : {
            "terms" : {
                "category.keyword" : cat
            }
        }
    }

    res = es.search(index=index, body=query, size=1000)['hits']['hits']
    results.extend(res)

In [16]:
len(results) == 473+38+29

True

In [37]:
len(dsmt_docs)

540

In [20]:
dsmt_docs = []
for res in results:
    _id = res['_id']
    source = res['_source']
    source_url = source['source_url']
    stored_url = source['stored_url']
    file_name = source['file_name']
    obj = {"_id": _id, "source_url": source_url, "stored_url": stored_url, "file_name": file_name}
    dsmt_docs.append(obj)

In [36]:
with open('DSMT-E_Document_Summary.jsonl','a') as f:
    for doc in dsmt_docs:
        f.write(json.dumps(doc)+'\n')

In [22]:
import boto3
import requests

profile = "wmuser"
bucket_name = "world-modelers"

session = boto3.Session(profile_name=profile)

s3 = session.resource("s3")
s3_client = boto3.client("s3")

bucket = s3.Bucket(bucket_name)

In [23]:
download_base_path = "/Users/brandon/Desktop/DSMT-Sample-Documents/"

In [34]:
count = 0
for doc in dsmt_docs:
    s3_key = doc['stored_url'].split('.com/')[1]
    file_name = download_base_path + doc['file_name']    
    bucket.download_file(s3_key, file_name)
    count += 1
    if count % 10 == 0:
        print(count)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
